<a href="https://colab.research.google.com/github/chunxu8894/tensorflow-tutorials/blob/master/Load_CSV_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("test.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


## 2. Load data

- load this using pandas and pass the Numpy arrays to Tensorflow.

- if you need to scale up to a large set of files, or need a loader that integrates with Tensorflow and tf.data the use the tf.data.experimental.make_csv_dataset function:

the only column you need to identify explicity is the one with the value that the model is intended to predict


In [3]:
LABEL_COLUMN = 'survived'
LABELS = [0,1]


In [4]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs
  )
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [5]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key, value.numpy()))

In [6]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [28. 28. 36. 16.  1.]
n_siblings_spouses  : [0 1 0 0 1]
parch               : [0 1 0 0 2]
fare                : [ 7.7292 15.2458  7.8958  7.7333 20.575 ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Cherbourg' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [7]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names = CSV_COLUMNS)
show_batch(temp_dataset)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [28. 29. 31. 64. 36.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 4 0 4 0]
fare                : [ 56.4958  21.075   10.5    263.      10.5   ]
class               : [b'Third' b'Third' b'Second' b'First' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'C' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'y']


## 3. Data preprocessing

A CSV file can contain a variety of data types. Typically you want to convert from those mixed types to a fixed length vector before feeding the data into your model

Tensorflow has a built-in system for describing common input conversions: __tf.feature_column__,

You can preprocess your data using any tool you like (like __ntlk__ or __sklearn__) and just pass the processed output to Tensorflow

The primary advantage of doing the preprocessing inside your model is that when you export the model it include the preprocessing. This way you can pass the raw data directly to your model


## 4. Continuous data

if your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model:


In [8]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0,0.0,0.0,0.0,0.0]

temp_dataset = get_dataset(train_file_path, select_columns = SELECT_COLUMNS, column_defaults = DEFAULTS)
show_batch(temp_dataset)

age                 : [ 9.  28.  24.  32.  32.5]
n_siblings_spouses  : [5. 1. 0. 0. 0.]
parch               : [2. 0. 0. 0. 0.]
fare                : [46.9     7.75    7.8958  7.8542 13.    ]


In [9]:
example_batch, labels_batch = next(iter(temp_dataset))

Here's a simple function that will pack together all the columns:

In [10]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [12]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[45.      0.      0.      6.975 ]
 [18.      0.      0.     13.    ]
 [30.      0.      0.     13.    ]
 [32.5     1.      0.     30.0708]
 [19.      0.      0.      7.65  ]]

[0 0 0 0 0]


if you have mixed data types you want to separate out these simple-numeric fields. The __tf.feature_column__ api can handle them, but this incurs some overhear and should be avoided unless really necessary. Switch back to the  mixed dataset:
  

In [13]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [22. 17. 28. 31. 28.]
n_siblings_spouses  : [0 0 0 1 1]
parch               : [1 0 0 0 1]
fare                : [55.      8.6625  7.775  18.     15.2458]
class               : [b'First' b'Third' b'Third' b'Third' b'Third']
deck                : [b'E' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'n' b'n']


In [14]:
example_batch, labels_batch = next(iter(temp_dataset))

In [15]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features
    return features, labels

In [16]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [17]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'female']
class               : [b'First' b'Third' b'First' b'Third' b'First']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'C']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'n']
numeric             : [[ 35.       1.       0.      90.    ]
 [ 26.       0.       0.       7.8875]
 [ 28.       0.       0.      39.6   ]
 [ 28.       8.       2.      69.55  ]
 [ 31.       0.       2.     164.8667]]


In [18]:
example_batch, labels_batch = next(iter(packed_train_data))

## 5. Data Normalization
Continous data should always be normalized


In [23]:
import pandas as pd

desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()

In [24]:
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [26]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [27]:
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

Now create a numeric column. The __tf.feature_columns.numeric_column__ API accepts a normalize_fn argument, which will be run on each batch

Bind the MEAN and STD to the normalizer fn using __functions.partial__

In [28]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
numeric_co

array([29.63130781,  0.54545455,  0.37958533, 34.38539856])